In [2]:
import pyreadr
import pandas as pd
import re
import string
from bertopic import BERTopic
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import random

# Step 0: Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')  # Optional: For extended WordNet data

[nltk_data] Downloading package stopwords to /Users/mw418/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/mw418/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/mw418/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
# Path to your .rds file
file_path = "/Users/mw418/codebase/covid_lit_analysis_transfer/data/bq_exports_mar_2023/clean/dat_main_abs.rds"

# Read the RDS file
result = pyreadr.read_r(file_path)


In [4]:
# Check the keys in the result
print("Keys:", result.keys())

Keys: odict_keys([None])


In [5]:
# Extract the DataFrame from the OrderedDict
df = list(result.values())[0]


In [6]:
# Display the first few rows
print(df.head())

# Check the DataFrame's shape
print(f"DataFrame shape: {df.shape}")

               id                                 abstract_preferred
0  pub.1129566575  2019年12月以来,湖北省武汉市陆续发现不明原因肺炎患者,经世界卫生组织确认其病原并命名为...
1  pub.1130421093  Resumo O presente artigo tem como objetivo ana...
2  pub.1133531192  Najpogostejši simptomi in znaki okužbe z virus...
3  pub.1128527039  Das neuartige Corona-Virus SarsCoV2 scheint na...
4  pub.1131251391  We must choose a personal not an impersonal fu...
DataFrame shape: (446818, 2)


In [7]:
# Check for missing values
missing_values = df['abstract_preferred'].isnull().sum()
print(f"Missing values in 'abstract_preferred': {missing_values}")

Missing values in 'abstract_preferred': 0


In [8]:
# Drop rows where 'abstract_preferred' is NaN
df = df.dropna(subset=['abstract_preferred'])

# Verify removal
print(f"DataFrame shape after dropping NaNs: {df.shape}")

DataFrame shape after dropping NaNs: (446818, 2)


In [9]:
# Convert to string if not already
df['abstract_preferred'] = df['abstract_preferred'].astype(str)

In [10]:
import re
import string

stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

df['abstract_cleaned'] = df['abstract_preferred'].apply(clean_text)


In [11]:

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

df['abstract_lemmatized'] = df['abstract_cleaned'].apply(lemmatize_text)
documents = df['abstract_lemmatized'].tolist()


In [12]:
# Initialize BERTopic
topic_model = BERTopic()

In [20]:
# Fit the model
# Use the first 10,000 abstracts for testing
# documents_subset = documents[:20000]
#topics_subset, probabilities_subset = topic_model.fit_transform(documents_subset)

topics_all, probabilities_all = topic_model.fit_transform(documents)

KeyboardInterrupt: 

In [18]:
# Visualize topics
fig = topic_model.visualize_topics()
fig.show()

